CONTAGION 
NODE BLOCKING
ERDOS-RENYI GRAPH


In [1]:
import numpy as np
import random
import pandas as pd

#!pip install pyomo
#import pyomo.environ as pyEnv
#!pip install -i https://pypi.gurobi.com gurobipy
import gurobipy as gp
from gurobipy import GRB
import copy
import time 

adjacency_df=pd.DataFrame(pd.read_csv("C:/Users/Lenovo/Desktop/adjacency_df-n100-m400-run1-risky40-inf10_utku.csv"))

n = len(adjacency_df)

adjacency_df.columns=(range(1,(n+4)))
adjacency_df=adjacency_df.rename(columns={n+1:'Threshold',n+2:'IsInfected', n+3:'IsRisky' },errors='raise')

adjacency_df.index=(range(1,(n+1)))

m=((adjacency_df.loc[:,range(1,n+1)]>0).sum().sum())/2

#sets
setV = range(1,n+1)
setS = adjacency_df.loc[(adjacency_df.loc[:, "IsInfected"] < 1),"IsInfected"].index.tolist()
setP = adjacency_df.loc[(adjacency_df.loc[:, "IsInfected"] > 0),"IsInfected"].index.tolist()

#parameters
wij = [[adjacency_df.loc[i,j] for i in setV] for j in setV  ]
ti = [adjacency_df.loc[i,"Threshold"] for i in setV]
Xi = [adjacency_df.loc[i,"IsRisky"] for i in setV]
#Blocking_Capacity = int((n) / 10) 
Blocking_Capacity = int((n) / 10)*3

start=time.time()
#model
Math_Model = gp.Model("Math_Model")

#variables
Yi = Math_Model.addVars(setV,vtype=GRB.BINARY,name="Yi") #Bunun yerine SetS'de tanımlı olabilir!!!!
Ii = Math_Model.addVars(setV,vtype=GRB.BINARY, obj=Xi, name="Ii")
Multi= Math_Model.addVars(setV,vtype=GRB.BINARY,name="Multi") #(1-Y[i])*(1-I[i])

#constraints
# Y[i] +I[i] <= 1   (1)
for i in setV:
    Math_Model.addConstr((Yi[i]+ Ii[i]), sense=GRB.LESS_EQUAL, rhs=1, name="Bloklanan birey infected olamaz".format(i))

    # Mult[i] =  (1-Y[i])*(1-I[i])   (1)
for i in setV:
    Math_Model.addConstr(Multi[i], sense=GRB.EQUAL, rhs=(1-Yi[i])*(1-Ii[i]), name="Multi constraint".format(i))#constraints
# Y[i] +I[i] <= 1   (1)
for i in setV:
    Math_Model.addConstr((Yi[i]+ Ii[i]), sense=GRB.LESS_EQUAL, rhs=1, name="Bloklanan birey infected olamaz".format(i))

#(Σ(over j in V  w[i][j]*I[j]))*(1-Y[i])*(1-I[i]) <=  t[i]    for all i in S
for i in setS:
    Math_Model.addConstr(lhs=gp.quicksum(wij[i-1][j-1]*Ii[j] for j in setV)*Multi[i] , sense=GRB.LESS_EQUAL, rhs=ti[i-1],name="Birinci:Tresholdu geçen birey hasta olur.".format(i))
#(Σ(over j in V  w[i][j]*I[j]))*(1-Y[i]) >  t[i]*I[i]         for all i in S 
for i in setS:
    Math_Model.addConstr(lhs=gp.quicksum(wij[i-1][j-1]*Ii[j] for j in setV)*(1-Yi[i]) , sense=GRB.GREATER_EQUAL, rhs=ti[i-1]*Ii[i],name="İkinci:Tresholdu geçen birey hasta olur.".format(i))

#Σ(over i Y[i])<= c 
Math_Model.addConstr(lhs=gp.quicksum(Yi[i] for i in setV) , sense=GRB.LESS_EQUAL, rhs=Blocking_Capacity,name="Bloklama kapasitesinden fazla bloklama yapılamaz.".format(i))

#I[i]=1       for all i in V-S(P) 
for i in setP:
    Math_Model.addConstr(Ii[i], sense=GRB.EQUAL, rhs=1, name="Başlangıçta enfekte olanların I[i] parametresi bire eşittir.".format(i))

    
    
Math_Model.write("out.lp")
# asd.computeIIS()
# asd.write("model.ilp")

Math_Model.ModelSense = GRB.MINIMIZE
Math_Model.optimize()

#for v in Math_Model.getVars():
    #if(v.x>= -0.0001):
        #print('%s %g' % (v.varName, v.x))

vars = Math_Model.getVars()

adjacency_df['Blocked']=np.zeros((n,1))

adjacency_df['Ending_Infected']=np.zeros((n,1))


for i in range(1,n+1):
    adjacency_df.at[i,'Blocked']=vars[i-1].x
    adjacency_df.at[i,'Ending_Infected']=vars[i+n-1].x

Objective_Value = Math_Model.ObjVal

#adjacency_df.loc[(adjacency_df.loc[:,'IsRisky'] == 1) & (adjacency_df.loc[:,'Ending_Infected'] == 1) ,['Threshold','IsInfected','IsRisky','Blocked','Ending_Infected']]

adjacency_df['Blocked'].to_csv("Math_Model_NODE-n"+str(n)+'-m'+str(m)+'-OBJ'+str(Objective_Value)+".csv",index=False)
end=time.time()





--------------------------------------------
--------------------------------------------

Academic license - for non-commercial use only - expires 2021-05-20
Using license file C:\Users\Lenovo\gurobi.lic
Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads
Optimize a model with 211 rows, 300 columns and 510 nonzeros
Model fingerprint: 0xe8e7b14f
Model has 280 quadratic constraints
Variable types: 0 continuous, 300 integer (300 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  QMatrix range    [1e-03, 1e+00]
  QLMatrix range   [1e-03, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 3e+01]
  QRHS range       [1e-02, 1e+00]
Presolve removed 53 rows and 76 columns
Presolve time: 0.01s
Presolved: 562 rows, 584 columns, 2435 nonzeros
Variable types: 0 continuous, 584 integer (584 binary)
Found heuristic solution: objective 26.0000000
Found 

In [2]:
end-start

0.6805238723754883

In [3]:
ati=adjacency_df['Blocked'].tolist()

In [4]:
adjacency_df['Blocked'].sum()

23.0

In [5]:
(adjacency_df.loc[adjacency_df['Ending_Infected']==1,'Blocked']).sum()

0.0

In [6]:
(adjacency_df.loc[adjacency_df['Ending_Infected']>0.1,'IsRisky']).sum()

0.0

In [7]:
len(adjacency_df)

100

In [8]:
adjacency_df.loc[adjacency_df['Blocked']==1,'IsRisky'].sum()

10.0

In [9]:
adjacency_df.loc[adjacency_df['Blocked']==1,setP]

,29,45,54,66,68,80,83,85,93,99
5,0.000,0.0,0.604,0.000,0.733,0.147,0.000,0.000,0.000,0.000
6,0.812,0.0,0.628,0.000,0.000,0.000,0.000,0.296,0.000,0.000
7,0.000,0.0,0.656,0.000,0.000,0.000,0.000,0.000,0.000,0.000
10,0.732,0.0,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
12,0.000,0.0,0.000,0.000,0.000,0.000,0.000,0.000,0.631,0.000
17,0.000,0.0,0.000,0.000,0.000,0.287,0.000,0.000,0.360,0.000
19,0.000,0.0,0.483,0.000,0.000,0.667,0.000,0.000,0.000,0.000
27,0.000,0.0,0.000,0.000,0.000,0.000,0.430,0.000,0.000,0.041
32,0.000,0.0,0.000,0.793,0.000,0.000,0.000,0.000,0.000,0.000
34,0.000,0.0,0.000,0.131,0.000,0.000,0.000,0.124,0.000,0.000


In [10]:
adjacency_df.loc[adjacency_df['Blocked']==1,setV]

,1,2,3,4,5,6,7,8,9,10,...,91,92,93,94,95,96,97,98,99,100
5,0.257,0.000,0.000,0.000,0.000,0.000,0.0,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.0,0.000,0.000
6,0.000,0.000,0.000,0.000,0.000,0.000,0.0,0.000,0.000,0.000,...,0.113,0.000,0.000,0.000,0.469,0.000,0.000,0.0,0.000,0.000
7,0.000,0.000,0.000,0.000,0.000,0.000,0.0,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.0,0.000,0.000
10,0.000,0.000,0.000,0.000,0.000,0.000,0.0,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.0,0.000,0.000
12,0.000,0.000,0.000,0.000,0.000,0.000,0.0,0.000,0.925,0.000,...,0.000,0.970,0.631,0.000,0.000,0.000,0.000,0.0,0.000,0.000
17,0.000,0.000,0.000,0.399,0.000,0.093,0.0,0.000,0.000,0.744,...,0.000,0.757,0.360,0.000,0.000,0.000,0.818,0.0,0.000,0.000
19,0.000,0.000,0.000,0.000,0.000,0.000,0.0,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.103,0.000,0.000,0.0,0.000,0.000
27,0.746,0.000,0.000,0.000,0.000,0.000,0.0,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.832,0.0,0.041,0.000
32,0.000,0.000,0.000,0.000,0.000,0.000,0.0,0.000,0.000,0.000,...,0.000,0.000,0.000,0.604,0.000,0.000,0.000,0.0,0.000,0.000
34,0.000,0.000,0.000,0.000,0.000,0.000,0.0,0.463,0.071,0.000,...,0.743,0.000,0.000,0.000,0.000,0.000,0.000,0.0,0.000,0.000


In [11]:
adjacency_df.loc[83,:]

1                  0.000
2                  0.000
3                  0.000
4                  0.000
5                  0.000
                   ...  
Threshold          0.975
IsInfected         1.000
IsRisky            0.000
Blocked            0.000
Ending_Infected    1.000
Name: 83, Length: 105, dtype: float64

In [12]:
adjacency_df.loc[adjacency_df['Blocked']==1,'Blocked']

5     1.0
6     1.0
7     1.0
10    1.0
12    1.0
17    1.0
19    1.0
27    1.0
32    1.0
34    1.0
36    1.0
39    1.0
41    1.0
44    1.0
65    1.0
71    1.0
73    1.0
77    1.0
79    1.0
87    1.0
88    1.0
91    1.0
96    1.0
Name: Blocked, dtype: float64